# Part 1: Read Data

In [10]:
import pandas as pd
import numpy as np
pd.set_option('chained_assignment', None)


lv = pd.read_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Enforcement Inspection Analysis\\jan-dec 2016\\Leave Requests Report.csv", low_memory=False)
ot = pd.read_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Enforcement Inspection Analysis\\overtime feb 2017 thru Jan 2017\\Overtime Requests Report.csv", low_memory=False)

ot["Name"] = ot["First Name"] + " " + ot["Last Name"]

ot = ot[["Approval Org/Command", "CityTime ID", "Last Name", "First Name","Name", "Rank", "Title", "OT Date","Start Time", "End Time", "Duration", "Voluntary/Involuntary"]]

ot["OT Date"] = pd.to_datetime(ot['OT Date'])
ot["Start Time"] = pd.to_datetime(ot['Start Time'])
ot["End Time"] = pd.to_datetime(ot['End Time'])


#clean up name
lv["First Name"] = lv["First Name"].map(str.upper)
lv["Last Name"] = lv["Last Name"].map(str.upper)

lv['Last Name'] = lv['Last Name'].str.replace(' JR', '')
lv['Last Name'] = lv['Last Name'].str.replace(',', '')
lv['Last Name'] = lv['Last Name'].str.replace('.', '')
lv['Last Name'] = lv['Last Name'].str.replace(' II', '')
lv['Last Name'] = lv['Last Name'].str.replace(' III', '')

lv["Name"] = lv["First Name"] + " " + lv["Last Name"]
lv["Name"] = lv["Name"].map(str.strip)


lv = lv[["Approval Org/Command", "CityTime ID", "Last Name", "First Name","Name", "Rank", "Title", "Leave Type","Start Time", "End Time", "Duration"]]


lv["Start Time"] = pd.to_datetime(lv['Start Time'])
lv["End Time"] = pd.to_datetime(lv['End Time'])

lv = lv.sort_values(by = 'Start Time', ascending=True).reset_index(drop=False)


lv["Start Month"] = ""
lv["End Month"] = ""
#get rid of start and end dates not in august
for i in range(0, len(lv)):
    lv["Start Month"][i] = lv["Start Time"][i].month
    lv["End Month"][i] = lv["End Time"][i].month
    
    

print "length of leave", len(lv)
#lv = lv[lv["Leave Type"] != "LATENESS"]
print "length of overtime", len(ot)

lv = lv.reset_index(drop=True)



length of leave 57485
length of overtime 102075


In [4]:
#get only those dates of interest:
#start_time = str(lv["Start Time"][i].month) + '-' + str(lv["Start Time"][i].day) + '-' + str(lv["Start Time"][i].year)
#end_time = str(lv["End Time"][i].month) + '-' + str(lv["End Time"][i].day) + '-' + str(lv["End Time"][i].year)

start_time = '1-1-2016'
end_time = '12-30-2016'
#subset the days worked using mask
mask = (lv['Start Time'] >= start_time) & (lv['End Time'] <= end_time)
lv = lv.loc[mask]

mask = (ot["OT Date"] >= start_time)
ot = ot.loc[mask]
ot = ot.reset_index(drop=True)


# Part 2: Overtime

In [11]:
print "Running overtime analysis ..."
print "Calculating overtime ..."

ot["Inspector"] = 0

ot["OT Month"] = "NA"
#create month variable
for i in range(0, len(ot)):
    ot["OT Month"][i] = ot["OT Date"][i].month


#convert to format so can get proper minutes, i.e. .30 is 30 minutues, not 3
ot["Duration"] = ot["Duration"].astype(str)

ot["Duration"] = ot["Duration"].str.replace('.', ':')

ot["Duration"] = ot["Duration"] + '0'


def HourConverter (a):
    a = str(a)
    a = a.split(":")
   
    hour = float(a[0])
    
    if len(a[1]) == 3:
        st = a[1]
        st = st[:-1]
        a[1] = st
    
    minutes = float(a[1])
    minutes_to_hours = float(minutes/60.0)
    
    total = hour + minutes_to_hours

   
    
    return total

ot["Duration"] = ot["Duration"].astype(str)

ot["Duration"] = ot["Duration"].str.replace('.', ':')

ot["Duration Hours"] = "NA"


for i in range(0, len(ot)):
    a = ot["Duration"][i]
    total_hours = HourConverter(a)
    ot["Duration Hours"][i] = total_hours
    

ot["Jan"] = 0.0
ot["Feb"] = 0.0
ot["Mar"] = 0.0
ot["Apr"] = 0.0

ot["May"] = 0.0
ot["Jun"] = 0.0
ot["Jul"] = 0.0
ot["Aug"] = 0.0

ot["Sep"] = 0.0
ot["Oct"] = 0.0

ot["Nov"] = 0.0

ot["Dec"] = 0.0

for i in range(0, len(ot)):
    if ot["OT Month"][i] == 1:
        ot["Jan"][i] = ot["Duration Hours"][i]

    if ot["OT Month"][i] == 2:
        ot["Feb"][i] = ot["Duration Hours"][i]
        
    if ot["OT Month"][i] == 3:
        ot["Mar"][i] = ot["Duration Hours"][i]
        
    if ot["OT Month"][i] == 4:
        ot["Apr"][i] = ot["Duration Hours"][i]
        
    if ot["OT Month"][i] == 5:
        ot["May"][i] = ot["Duration Hours"][i]
    
    if ot["OT Month"][i] == 6:
        ot["Jun"][i] = ot["Duration Hours"][i] 
        
    if ot["OT Month"][i] == 7:
        ot["Jul"][i] = ot["Duration Hours"][i]   
    
    if ot["OT Month"][i] == 8:
        ot["Aug"][i] = ot["Duration Hours"][i] 
        
    if ot["OT Month"][i] == 9:
        ot["Sep"][i] = ot["Duration Hours"][i]
        
    if ot["OT Month"][i] == 10:
        ot["Oct"][i] = ot["Duration Hours"][i] 
    
    if ot["OT Month"][i] == 11:
        ot["Nov"][i] = ot["Duration Hours"][i] 
    
    if ot["OT Month"][i] == 12:
        ot["Dec"][i] = ot["Duration Hours"][i] 
        
ot['Jan'] = ot['Jan'].groupby(ot['CityTime ID']).transform('sum')
ot['Feb'] = ot['Feb'].groupby(ot['CityTime ID']).transform('sum')


ot['Mar'] = ot['Mar'].groupby(ot['CityTime ID']).transform('sum')
ot['Apr'] = ot['Apr'].groupby(ot['CityTime ID']).transform('sum')

ot['May'] = ot['May'].groupby(ot['CityTime ID']).transform('sum')
ot['Jun'] = ot['Jun'].groupby(ot['CityTime ID']).transform('sum')

ot['Jul'] = ot['Jul'].groupby(ot['CityTime ID']).transform('sum')
ot['Aug'] = ot['Aug'].groupby(ot['CityTime ID']).transform('sum')

ot['Sep'] = ot['Sep'].groupby(ot['CityTime ID']).transform('sum')

ot['Oct'] = ot['Oct'].groupby(ot['CityTime ID']).transform('sum')

ot['Nov'] = ot['Nov'].groupby(ot['CityTime ID']).transform('sum')

ot['Dec'] = ot['Dec'].groupby(ot['CityTime ID']).transform('sum')


ot = ot.drop_duplicates(['CityTime ID']).reset_index(drop=True)

ot["Jan"] = ot["Jan"].round(2)
ot["Feb"] = ot["Feb"].round(2)
ot["Mar"] = ot["Mar"].round(2)
ot["Apr"] = ot["Apr"].round(2)
ot["May"] = ot["May"].round(2)
ot["Jun"] = ot["Jun"].round(2)
ot["Jul"] = ot["Jul"].round(2)
ot["Aug"] = ot["Aug"].round(2)
ot["Sep"] = ot["Sep"].round(2)
ot["Oct"] = ot["Oct"].round(2)
ot["Nov"] = ot["Nov"].round(2)
ot["Dec"] = ot["Dec"].round(2)

#Classify as inspector

print "Classifying inspectors ..."
bn = pd.read_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Enforcement Inspection Analysis\\Inspector List\\Inspector Headcount (9-16-16).csv")
bn["FIRST NAME"] = bn["FIRST NAME"].map(str.upper)
bn["LAST NAME"] = bn["LAST NAME"].map(str.upper)
bn["FIRST NAME"] = bn["FIRST NAME"].map(str.strip)
bn["LAST NAME"] = bn["LAST NAME"].map(str.strip)

bn["LAST NAME"] = bn["LAST NAME"].str.replace(' JR', '')
bn["LAST NAME"] = bn["LAST NAME"].str.replace(',', '')
bn["LAST NAME"] = bn["LAST NAME"].str.replace('.', '')
bn["LAST NAME"] = bn["LAST NAME"].str.replace(' II', '')
bn["LAST NAME"] = bn["LAST NAME"].str.replace(' III', '')

bn["Name"] = bn["FIRST NAME"] + " " + bn["LAST NAME"]


for i in range(0, len(bn)):
    if ("Development Hub" in bn["UNIT"][i] and "Electrical" in bn["OFFICE TITLE (DOB)"][i]):
        bn["UNIT"][i] = "Electical Development"
        
#list_insp = []

list_insp = {}

ot = ot.reset_index(drop=True)
ot["Unit"] = ""

for i in range(0, len(bn)):
    list_insp.update({bn["Name"][i]:bn["UNIT"][i]})

c=0
for i in range(0, len(ot)): 
    if ot["Name"][i] in list_insp:
        ot["Inspector"][i]  = 1
        ot["Unit"][i] = list_insp[ot["Name"][i]]
        c+=1

print "Number of inspectors who took overtime", c
print "DONE"

Running overtime analysis ...
Calculating overtime ...
Classifying inspectors ...
Number of inspectors who took overtime 379
DONE


In [13]:
#Export
ot = ot.drop('Approval Org/Command', 1)
ot = ot.drop('Name', 1)
ot = ot.drop('OT Date', 1)
ot = ot.drop('Start Time', 1)
ot = ot.drop('End Time', 1)
ot = ot.drop('Duration', 1)
ot = ot.drop('Voluntary/Involuntary', 1)
ot = ot.drop('OT Month', 1)
ot = ot.drop('Duration Hours', 1)
ot.to_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Enforcement Inspection Analysis\\output\\OVERTIME FEB 16 - JAN 17.csv")

# Part 3: CLASSIFY LEAVE DATASET AS INSPECTOR

In [10]:
print "Classifying inspectors ..."
bn = pd.read_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Enforcement Inspection Analysis\\Inspector List\\Inspector Headcount (9-16-16).csv")

#INSERT INSPECTOR NAMES

bn["FIRST NAME"] = bn["FIRST NAME"].map(str.upper)
bn["LAST NAME"] = bn["LAST NAME"].map(str.upper)
lv["First Name"] = lv["First Name"].map(str.upper)
lv["Last Name"] = lv["Last Name"].map(str.upper)

bn["FIRST NAME"] = bn["FIRST NAME"].map(str.strip)
bn["LAST NAME"] = bn["LAST NAME"].map(str.strip)

lv["First Name"] = lv["First Name"].map(str.strip)
lv["Last Name"] = lv["Last Name"].map(str.strip)

bn["LAST NAME"] = bn["LAST NAME"].str.replace(' JR', '')
bn["LAST NAME"] = bn["LAST NAME"].str.replace(',', '')
bn["LAST NAME"] = bn["LAST NAME"].str.replace('.', '')
bn["LAST NAME"] = bn["LAST NAME"].str.replace(' II', '')
bn["LAST NAME"] = bn["LAST NAME"].str.replace(' III', '')

bn["Name"] = bn["FIRST NAME"] + " " + bn["LAST NAME"]

lv["Inspector"] = 0

list_insp = []

lv = lv.reset_index(drop=True)


for i in range(0, len(bn)):
    list_insp.append(bn["Name"][i])

c=0
for i in range(0, len(lv)): 
    if lv["Name"][i] in list_insp:
        lv["Inspector"][i]  = 1
        c+=1


print 'total number of employees', len(lv)

#lv = lv[lv["Inspector"] == 1]
#lv = lv.reset_index(drop=True)


print "number of inspectors", c

print "DONE"

#FOLLOW UP WITH ANOTHER INSPECTION CHECK USING TITLE IF LEAVE DATASET


Classifying inspectors ...
total number of employees 55437
number of inspectors 13826
DONE


# Part 4: DATE TIME CORRECTION DICTIONARY

In [11]:
wd_list = [
            #2015
            #jan
            '2015-01-2',
    
            '2015-01-5', 
            '2015-01-6',
            '2015-01-7',
            '2015-01-8',
            '2015-01-9',
    
            '2015-01-12', 
            '2015-01-13',
            '2015-01-14',
            '2015-01-15',
            '2015-01-16',
    
            '2015-01-20', 
            '2015-01-21',
            '2015-01-22',
            '2015-01-23',
            
            '2015-01-26', 
            '2015-01-27',
            '2015-01-28',
            '2015-01-29',
            '2015-01-30',
            
            #feb
            '2015-02-2', 
            '2015-02-3',
            '2015-02-4',
            '2015-02-5',
            '2015-02-6',
            
            '2015-02-9', 
            '2015-02-10',
            '2015-02-11',
            '2015-02-13',
            
            '2015-02-16', 
            '2015-02-17',
            '2015-02-18',
            '2015-02-19',
            '2015-02-20',
    
            '2015-02-23', 
            '2015-02-24',
            '2015-02-25',
            '2015-02-26',
            '2015-02-27',
    
            #march
            '2015-03-2', 
            '2015-03-3',
            '2015-03-4',
            '2015-03-5',
            '2015-03-6',
    
            '2015-03-9', 
            '2015-03-10',
            '2015-03-11',
            '2015-03-12',
            '2015-03-13',
    
            '2015-03-16', 
            '2015-03-17',
            '2015-03-18',
            '2015-03-19',
            '2015-03-20',
            
            '2015-03-23', 
            '2015-03-24',
            '2015-03-25',
            '2015-03-26',
            '2015-03-27',
    
            '2015-03-30',
            '2015-03-31',
    
            #april
            '2015-04-1', 
            '2015-04-2',
    
            '2015-04-6',
            '2015-04-7',
            '2015-04-8',
            '2015-04-9',
            '2015-04-10',
    
            '2015-04-13',
            '2015-04-14',
            '2015-04-15',
            '2015-04-16',
            '2015-04-17',
    
            '2015-04-20',
            '2015-04-21',
            '2015-04-22',
            '2015-04-23',
            '2015-04-24',
            
            '2015-04-27',
            '2015-04-28',
            '2015-04-29',
            '2015-04-30',
    
            #may
            '2015-05-1', 
           
            '2015-05-4',
            '2015-05-5',
            '2015-05-6',
            '2015-05-7',
            '2015-05-8',
    
            '2015-05-11',
            '2015-05-12',
            '2015-05-13',
            '2015-05-14',
            '2015-05-15',
    
            '2015-05-18',
            '2015-05-19',
            '2015-05-20',
            '2015-05-21',
            '2015-05-22',
    
            '2015-05-26',
            '2015-05-27',
            '2015-05-28',
            '2015-05-29',
    
            #june
            '2015-06-1',
            '2015-06-2',
            '2015-06-3',
            '2015-06-4',
            '2015-06-5',
            
            '2015-06-8',
            '2015-06-9',
            '2015-06-10',
            '2015-06-11',
            '2015-06-12',
    
            '2015-06-15',
            '2015-06-16',
            '2015-06-17',
            '2015-06-18',
            '2015-06-19',
            
            
            '2015-06-22',
            '2015-06-23',
            '2015-06-24',
            '2015-06-25',
            '2015-06-26',
    
            '2015-06-29',
            '2015-06-30',
    
             #july
            '2015-07-1',
            '2015-07-2',
    
            '2015-07-6',
            '2015-07-7',
            '2015-07-8',
            '2015-07-9',
            '2015-07-10',
    
            '2015-07-13',
            '2015-07-14',
            '2015-07-15',
            '2015-07-16',
            '2015-07-17',
    
            '2015-07-20',
            '2015-07-21',
            '2015-07-22',
            '2015-07-23',
            '2015-07-24',
    
            '2015-07-27',
            '2015-07-28',
            '2015-07-29',
            '2015-07-30',
            '2015-07-31',
    
            #august
            '2015-08-3',
            '2015-08-4',
            '2015-08-5',
            '2015-08-6',
            '2015-08-7',
    
            '2015-08-10',
            '2015-08-11',
            '2015-08-12',
            '2015-08-13',
            '2015-08-14',
    
            '2015-08-17',
            '2015-08-18',
            '2015-08-19',
            '2015-08-20',
            '2015-08-21',
    
            '2015-08-24',
            '2015-08-25',
            '2015-08-26',
            '2015-08-27',
            '2015-08-28',
    
            '2015-08-31',
    
            #sept
            '2015-09-1',
            '2015-09-2',
            '2015-09-3',
            '2015-09-4',
    
            '2015-09-8',
            '2015-09-9',
            '2015-09-10',
            '2015-09-11',
    
            '2015-09-14',
            '2015-09-15',
            '2015-09-16',
            '2015-09-17',
            '2015-09-18',
    
            '2015-09-21',
            '2015-09-22',
            '2015-09-23',
            '2015-09-24',
            '2015-09-25',
    
            '2015-09-28',
            '2015-09-29',
            '2015-09-30',
            
            #oct
            '2015-10-1',
            '2015-10-2',
    
            '2015-10-5',
            '2015-10-6',
            '2015-10-7',
            '2015-10-8',
            '2015-10-9',
    
            '2015-10-13',
            '2015-10-14',
            '2015-10-15',
            '2015-10-16',
            
            '2015-10-19',
            '2015-10-20',
            '2015-10-21',
            '2015-10-22',
            '2015-10-23',
            
            '2015-10-26',
            '2015-10-27',
            '2015-10-28',
            '2015-10-29',
            '2015-10-30',
    
            #nov
            '2015-11-2',
            '2015-11-4',
            '2015-11-5',
            '2015-11-6',
    
            '2015-11-9',
            '2015-11-10',
            '2015-11-12',
            '2015-11-13',
    
            '2015-11-16',
            '2015-11-17',
            '2015-11-18',
            '2015-11-19',
            '2015-11-20',
    
            '2015-11-23',
            '2015-11-24',
            '2015-11-25',
            '2015-11-27',
            
            '2015-11-30',
    
            #dec
            '2015-12-1',
            '2015-12-2',
            '2015-12-3',
            '2015-12-4',
    
            '2015-12-7',
            '2015-12-8',
            '2015-12-9',
            '2015-12-10',
            '2015-12-11',

            '2015-12-14',
            '2015-12-15',
            '2015-12-16',
            '2015-12-17',
            '2015-12-18',
    
            '2015-12-21',
            '2015-12-22',
            '2015-12-23',
            '2015-12-24',
          
            '2015-12-28',
            '2015-12-29',
            '2015-12-30',
            '2015-12-31',
    
            #***********
            #2016
            #jan
            '2016-01-4', 
            '2016-01-5', 
            '2016-01-6',
            '2016-01-7',
            '2016-01-8',
    
            '2016-01-11', 
            '2016-01-12', 
            '2016-01-13',
            '2016-01-14',
            '2016-01-15',
    
            '2016-01-19', 
            '2016-01-20', 
            '2016-01-21',
            '2016-01-22',
            
            '2016-01-25', 
            '2016-01-26', 
            '2016-01-27',
            '2016-01-28',
            '2016-01-29',
    
            #feb
            '2016-02-1', 
            '2016-02-2', 
            '2016-02-3',
            '2016-02-4',
            '2016-02-5',
    
            '2016-02-8', 
            '2016-02-9', 
            '2016-02-10',
            '2016-02-11',
            '2016-02-12',
    
            '2016-02-16', 
            '2016-02-17',
            '2016-02-18',
            '2016-02-19',
    
            '2016-02-22', 
            '2016-02-23', 
            '2016-02-24',
            '2016-02-25',
            '2016-02-26',
            
            '2016-02-29',
    
            #march
            '2016-03-1', 
            '2016-03-2', 
            '2016-03-3',
            '2016-03-4',
    
            '2016-03-7', 
            '2016-03-8', 
            '2016-03-9',
            '2016-03-10',
            '2016-03-11',
    
            '2016-03-14', 
            '2016-03-15', 
            '2016-03-16',
            '2016-03-17',
            '2016-03-18',
    
            '2016-03-21', 
            '2016-03-22', 
            '2016-03-23',
            '2016-03-24',
            '2016-03-25',
    
            '2016-03-28', 
            '2016-03-29', 
            '2016-03-30',
            '2016-03-31',
    
            #april
            '2016-04-1', 
    
            '2016-04-4', 
            '2016-04-5', 
            '2016-04-6',
            '2016-04-7',
            '2016-04-8',
    
            '2016-04-11', 
            '2016-04-12', 
            '2016-04-13',
            '2016-04-14',
            '2016-04-15',
            
            '2016-04-18', 
            '2016-04-19', 
            '2016-04-20',
            '2016-04-21',
            '2016-04-22',
            
            '2016-04-25', 
            '2016-04-26', 
            '2016-04-27',
            '2016-04-28',
            '2016-04-29',
    
            #may
            '2016-05-2', 
            '2016-05-3', 
            '2016-05-4',
            '2016-05-5',
            '2016-05-6',
    
            '2016-05-9', 
            '2016-05-10', 
            '2016-05-11',
            '2016-05-12',
            '2016-05-13',
    
            '2016-05-16', 
            '2016-05-17', 
            '2016-05-18',
            '2016-05-19',
            '2016-05-20',
    
            '2016-05-23', 
            '2016-05-24', 
            '2016-05-25',
            '2016-05-26',
            '2016-05-27',
    
            '2016-05-31',
    
            #june
            '2016-06-1', 
            '2016-06-2', 
            '2016-06-3',

            '2016-06-6', 
            '2016-06-7', 
            '2016-06-8',
            '2016-06-9',
            '2016-06-10',
    
            '2016-06-13', 
            '2016-06-14', 
            '2016-06-15',
            '2016-06-16',
            '2016-06-17',
            
            '2016-06-20', 
            '2016-06-21', 
            '2016-06-22',
            '2016-06-23',
            '2016-06-24',
    
            '2016-06-27', 
            '2016-06-28', 
            '2016-06-29',
            '2016-06-30',
            
    
            #july
            '2016-07-1',
    
            '2016-07-5', 
            '2016-07-6', 
            '2016-07-7',
            '2016-07-8',
            
    
            '2016-07-11', 
            '2016-07-12', 
            '2016-07-13',
            '2016-07-14',
            '2016-07-15',
            
            '2016-07-18', 
            '2016-07-19', 
            '2016-07-20',
            '2016-07-21',
            '2016-07-22',
    
            '2016-07-25', 
            '2016-07-26', 
            '2016-07-27',
            '2016-07-28',
            '2016-07-29',
           
           #august
            '2016-08-1',
            '2016-08-2',
            '2016-08-3',
            '2016-08-4',
            '2016-08-5',
           
            '2016-08-8',
            '2016-08-9',
            '2016-08-10',
            '2016-08-11',
            '2016-08-12',
           
            '2016-08-15',
            '2016-08-16',
            '2016-08-17',
            '2016-08-18',
            '2016-08-19',
           
            '2016-08-22',
            '2016-08-23',
            '2016-08-24',
            '2016-08-25',
            '2016-08-26',
           
            '2016-08-29',
            '2016-08-30',
            '2016-08-31',
           
           #september
            '2016-09-1',
            '2016-09-2',
           
            '2016-09-6',
            '2016-09-7',
            '2016-09-8',
            '2016-09-9',
           
            '2016-09-12',
            '2016-09-13',
            '2016-09-14',
            '2016-09-15',
            '2016-09-16',
           
            '2016-09-19',
            '2016-09-20',
            '2016-09-21',
            '2016-09-22',
            '2016-09-23',
           
            '2016-09-26',
            '2016-09-27',
            '2016-09-28',
            '2016-09-29',
            '2016-09-30',
    
            #october
            '2016-10-3',
            '2016-10-4',
            '2016-10-5',
            '2016-10-6',
            '2016-10-7',
           
            '2016-10-11',
            '2016-10-12',
            '2016-10-13',
            '2016-10-14',
            
            '2016-10-17',
            '2016-10-18',
            '2016-10-19',
            '2016-10-20',
            '2016-10-21',
    
            '2016-10-24',
            '2016-10-25',
            '2016-10-26',
            '2016-10-27',
            '2016-10-28',
    
            '2016-10-31',
    
            #november
            '2016-11-1',
            '2016-11-2',
            '2016-11-3',
            '2016-11-4',
            
            '2016-11-7',
            '2016-11-9',
            '2016-11-10',
            
            '2016-11-14',
            '2016-11-15',
            '2016-11-16',
            '2016-11-17',
            '2016-11-18',

            '2016-11-21',
            '2016-11-22',
            '2016-11-23',
            '2016-11-25',
    
            '2016-11-28',
            '2016-11-29',
            '2016-11-30',
    
            #december
            '2016-12-1',
            '2016-12-2',
            
            '2016-12-5',
            '2016-12-6',
            '2016-12-7',
            '2016-12-8',
            '2016-12-9',
    
            '2016-12-12',
            '2016-12-13',
            '2016-12-14',
            '2016-12-15',
            '2016-12-16',
            
            '2016-12-19',
            '2016-12-20',
            '2016-12-21',
            '2016-12-22',
            '2016-12-23',
            
            '2016-12-27',
            '2016-12-28',
            '2016-12-29',
    
            
    
            
          ]

wd = pd.DataFrame({'workdays': wd_list })
wd['workdays'] = pd.to_datetime(wd['workdays'])

#wd['day'] = 1
#wd

# Part 5: CALCULATE CORRECT LEAVE TIME

In [12]:
#TIME CORRECTION
print "Running"

import datetime

pd.set_option('chained_assignment', None)
count_same_day = 0

#minute to hour time converter
def HourConverter (a):
    a = str(a)
    a = a.split(":")
    #print ""
    #print i
    #print a

    hour = float(a[0])
    #print hour
    minutes = float(a[1])
    

    minutes_to_hours = float(minutes/60.0)
    #print minutes_to_hours

    total = hour + minutes_to_hours

    #print "total", total
    
    return total
    


lv["C Jan"] = 0.0
lv["C Feb"] = 0.0
lv["C Mar"] = 0.0
lv["C Apr"] = 0.0
lv["C May"] = 0.0
lv["C Jun"] = 0.0
lv["C Jul"] = 0.0
lv["C Aug"] = 0.0
lv["C Sep"] = 0.0
lv["C Oct"] = 0.0
lv["C Nov"] = 0.0
lv["C Dec"] = 0.0

lv = lv.reset_index(drop=True)


for j in range(1, 13):
    
    print "j = ", j
    
    for i in range(1, len(lv)):
        if i%10000 == 0:
            print i
        
        start_time = str(lv["Start Time"][i].month) + '-' + str(lv["Start Time"][i].day) + '-' + str(lv["Start Time"][i].year)
        end_time = str(lv["End Time"][i].month) + '-' + str(lv["End Time"][i].day) + '-' + str(lv["End Time"][i].year)

        #subset the days worked using mask
        mask = (wd['workdays'] >= start_time) & (wd['workdays'] <= end_time)
        wd_subset = wd.loc[mask]


        if j == 1:
            #subset for month, calculate hours only for this month
            wd_subset = wd_subset[wd_subset.workdays.dt.month == j]
            days_leave = len(wd_subset)
            if lv["Inspector"][i] == 0:
                hours_leave = 7*days_leave
            elif lv["Inspector"][i] == 1:
                hours_leave = 8*days_leave

            lv["C Jan"][i] = hours_leave

            #correct if leave is on same day
            if start_time == end_time:

                a = lv["Duration"][i]

                lv["C Jan"][i] = HourConverter (a)
                #print lv["C Jan"][i]

                if (lv["Start Month"][i] > j and lv["End Month"][i] > j):
                    lv["C Jan"][i] = 0.0
                        #if (lv["Start Month"][i] < j and lv["End Month"][i] < j): 
                            #lv["C Jan"][i] = 0
       
        if j == 2:
            #subset for month, calculate hours only for this month
            wd_subset = wd_subset[wd_subset.workdays.dt.month == j]
            days_leave = len(wd_subset)
            if lv["Inspector"][i] == 0:
                hours_leave = 7*days_leave
            elif lv["Inspector"][i] == 1:
                hours_leave = 8*days_leave

            lv["C Feb"][i] = hours_leave

            #correct if leave is on same day
            if start_time == end_time:

                a = lv["Duration"][i]

                lv["C Feb"][i] = HourConverter (a)
                #print lv["C Feb"][i]

                if (lv["Start Month"][i] > j and lv["End Month"][i] > j):
                    lv["C Feb"][i] = 0.0
                if (lv["Start Month"][i] < j and lv["End Month"][i] < j): 
                    lv["C Feb"][i] = 0.0
        if j == 3:
            #subset for month, calculate hours only for this month
            wd_subset = wd_subset[wd_subset.workdays.dt.month == j]
            days_leave = len(wd_subset)
            if lv["Inspector"][i] == 0:
                hours_leave = 7*days_leave
            elif lv["Inspector"][i] == 1:
                hours_leave = 8*days_leave

            lv["C Mar"][i] = hours_leave

            #correct if leave is on same day
            if start_time == end_time: 

                a = lv["Duration"][i]

                lv["C Mar"][i] = HourConverter (a)
                #print lv["C Feb"][i]

                if (lv["Start Month"][i] > j and lv["End Month"][i] > j):
                    lv["C Mar"][i] = 0.0
                if (lv["Start Month"][i] < j and lv["End Month"][i] < j): 
                    lv["C Mar"][i] = 0.0
        if j == 4:
            #subset for month, calculate hours only for this month
            wd_subset = wd_subset[wd_subset.workdays.dt.month == j]
            days_leave = len(wd_subset)
            if lv["Inspector"][i] == 0:
                hours_leave = 7*days_leave
            elif lv["Inspector"][i] == 1:
                hours_leave = 8*days_leave

            lv["C Apr"][i] = hours_leave

            #correct if leave is on same day
            if start_time == end_time: 

                a = lv["Duration"][i]

                lv["C Apr"][i] = HourConverter (a)
                #print lv["C Feb"][i]

                if (lv["Start Month"][i] > j and lv["End Month"][i] > j):
                    lv["C Apr"][i] = 0.0
                if (lv["Start Month"][i] < j and lv["End Month"][i] < j): 
                    lv["C Apr"][i] = 0.0

        if j == 5:
            #subset for month, calculate hours only for this month
            wd_subset = wd_subset[wd_subset.workdays.dt.month == j]
            days_leave = len(wd_subset)
            if lv["Inspector"][i] == 0:
                hours_leave = 7*days_leave
            elif lv["Inspector"][i] == 1:
                hours_leave = 8*days_leave

            lv["C May"][i] = hours_leave

            #correct if leave is on same day
            if start_time == end_time: 

                a = lv["Duration"][i]

                lv["C May"][i] = HourConverter (a)
                #print lv["C Feb"][i]

                if (lv["Start Month"][i] > j and lv["End Month"][i] > j):
                    lv["C May"][i] = 0.0
                if (lv["Start Month"][i] < j and lv["End Month"][i] < j): 
                    lv["C May"][i] = 0.0
        if j == 6:
            #subset for month, calculate hours only for this month
            wd_subset = wd_subset[wd_subset.workdays.dt.month == j]
            days_leave = len(wd_subset)
            if lv["Inspector"][i] == 0:
                hours_leave = 7*days_leave
            elif lv["Inspector"][i] == 1:
                hours_leave = 8*days_leave

            lv["C Jun"][i] = hours_leave

            #correct if leave is on same day
            if start_time == end_time: 

                a = lv["Duration"][i]

                lv["C Jun"][i] = HourConverter (a)
                #print lv["C Feb"][i]

                if (lv["Start Month"][i] > j and lv["End Month"][i] > j):
                    lv["C Jun"][i] = 0.0
                if (lv["Start Month"][i] < j and lv["End Month"][i] < j): 
                    lv["C Jun"][i] = 0.0
        if j == 7:
            #subset for month, calculate hours only for this month
            wd_subset = wd_subset[wd_subset.workdays.dt.month == j]
            days_leave = len(wd_subset)
            if lv["Inspector"][i] == 0:
                hours_leave = 7*days_leave
            elif lv["Inspector"][i] == 1:
                hours_leave = 8*days_leave

            lv["C Jul"][i] = hours_leave

            #correct if leave is on same day
            if start_time == end_time: 

                a = lv["Duration"][i]

                lv["C Jul"][i] = HourConverter (a)
                #print lv["C Feb"][i]

                if (lv["Start Month"][i] > j and lv["End Month"][i] > j):
                    lv["C Jul"][i] = 0.0
                if (lv["Start Month"][i] < j and lv["End Month"][i] < j): 
                    lv["C Jul"][i] = 0.0
        if j == 8:
            #subset for month, calculate hours only for this month
            wd_subset = wd_subset[wd_subset.workdays.dt.month == j]
            days_leave = len(wd_subset)
            if lv["Inspector"][i] == 0:
                hours_leave = 7*days_leave
            elif lv["Inspector"][i] == 1:
                hours_leave = 8*days_leave

            lv["C Aug"][i] = hours_leave

            #correct if leave is on same day
            if start_time == end_time: 

                a = lv["Duration"][i]

                lv["C Aug"][i] = HourConverter (a)
                #print lv["C Feb"][i]

                if (lv["Start Month"][i] > j and lv["End Month"][i] > j):
                    lv["C Aug"][i] = 0.0
                if (lv["Start Month"][i] < j and lv["End Month"][i] < j): 
                    lv["C Aug"][i] = 0.0  
        if j == 9:
            #subset for month, calculate hours only for this month
            wd_subset = wd_subset[wd_subset.workdays.dt.month == j]
            days_leave = len(wd_subset)
            if lv["Inspector"][i] == 0:
                hours_leave = 7*days_leave
            elif lv["Inspector"][i] == 1:
                hours_leave = 8*days_leave

            lv["C Sep"][i] = hours_leave

            #correct if leave is on same day
            if start_time == end_time: 

                a = lv["Duration"][i]

                lv["C Sep"][i] = HourConverter (a)
                #print lv["C Feb"][i]

                if (lv["Start Month"][i] > j and lv["End Month"][i] > j):
                    lv["C Sep"][i] = 0.0
                if (lv["Start Month"][i] < j and lv["End Month"][i] < j): 
                    lv["C Sep"][i] = 0.0  
          
        if j == 10:
            #subset for month, calculate hours only for this month
            wd_subset = wd_subset[wd_subset.workdays.dt.month == j]
            days_leave = len(wd_subset)
            if lv["Inspector"][i] == 0:
                hours_leave = 7*days_leave
            elif lv["Inspector"][i] == 1:
                hours_leave = 8*days_leave

            lv["C Oct"][i] = hours_leave

            #correct if leave is on same day
            if start_time == end_time: 

                a = lv["Duration"][i]

                lv["C Oct"][i] = HourConverter (a)
                #print lv["C Feb"][i]

                if (lv["Start Month"][i] > j and lv["End Month"][i] > j):
                    lv["C Oct"][i] = 0.0
                if (lv["Start Month"][i] < j and lv["End Month"][i] < j): 
                    lv["C Oct"][i] = 0.0  
    
        if j == 11:
            #subset for month, calculate hours only for this month
            wd_subset = wd_subset[wd_subset.workdays.dt.month == j]
            days_leave = len(wd_subset)
            if lv["Inspector"][i] == 0:
                hours_leave = 7*days_leave
            elif lv["Inspector"][i] == 1:
                hours_leave = 8*days_leave

            lv["C Nov"][i] = hours_leave

            #correct if leave is on same day
            if start_time == end_time: 

                a = lv["Duration"][i]

                lv["C Nov"][i] = HourConverter (a)
                #print lv["C Feb"][i]

                if (lv["Start Month"][i] > j and lv["End Month"][i] > j):
                    lv["C Nov"][i] = 0.0
                if (lv["Start Month"][i] < j and lv["End Month"][i] < j): 
                    lv["C Nov"][i] = 0.0  
        if j == 12:
            #subset for month, calculate hours only for this month
            wd_subset = wd_subset[wd_subset.workdays.dt.month == j]
            days_leave = len(wd_subset)
            if lv["Inspector"][i] == 0:
                hours_leave = 7*days_leave
            elif lv["Inspector"][i] == 1:
                hours_leave = 8*days_leave

            lv["C Dec"][i] = hours_leave

            #correct if leave is on same day
            if start_time == end_time: 

                a = lv["Duration"][i]

                lv["C Dec"][i] = HourConverter (a)
                #print lv["C Feb"][i]

                if (lv["Start Month"][i] > j and lv["End Month"][i] > j):
                    lv["C Dec"][i] = 0.0
                if (lv["Start Month"][i] < j and lv["End Month"][i] < j): 
                    lv["C Dec"][i] = 0.0  
        
lv["C Jan"] = lv["C Jan"].astype(float)
lv["C Jan"] = lv["C Jan"].round(2)
lv["C Feb"] = lv["C Feb"].astype(float)
lv["C Feb"] = lv["C Feb"].round(2)

lv["C Mar"] = lv["C Mar"].astype(float)
lv["C Mar"] = lv["C Mar"].round(2)
lv["C Apr"] = lv["C Apr"].astype(float)
lv["C Apr"] = lv["C Apr"].round(2)

lv["C May"] = lv["C May"].round(2)
lv["C Jun"] = lv["C Jun"].round(2)
lv["C Jul"] = lv["C Jul"].round(2)
lv["C Aug"] = lv["C Aug"].round(2)
lv["C Sep"] = lv["C Sep"].round(2)
lv["C Oct"] = lv["C Oct"].round(2)
lv["C Nov"] = lv["C Nov"].round(2)

lv["C Dec"] = lv["C Dec"].round(2)

#lv["C Feb"] = lv["C Feb"].astype(float)
#lv["C Feb"] = lv["C Feb"].round(2)


print "number of employees on leave ", len(lv)
print "DONE"

Running
j =  1
10000
20000
30000
40000
50000
j =  2
10000
20000
30000
40000
50000
j =  3
10000
20000
30000
40000
50000
j =  4
10000
20000
30000
40000
50000
j =  5
10000
20000
30000
40000
50000
j =  6
10000
20000
30000
40000
50000
j =  7
10000
20000
30000
40000
50000
j =  8
10000
20000
30000
40000
50000
j =  9
10000
20000
30000
40000
50000
j =  10
10000
20000
30000
40000
50000
j =  11
10000
20000
30000
40000
50000
j =  12
10000
20000
30000
40000
50000
number of employees on leave  55437
DONE


# Part 6: CALCULATE AND EXPORT ANNUAL LEAVE

In [13]:
al = lv
            
al = al[al['Leave Type'] == 'ANNUAL LEAVE']
al = al.reset_index(drop=True)


#ADD TOTAL LEAVE HOURS PER PERSON AND THEN DROP DUPLICATES
al['C Jan Sum'] = al['C Jan'].groupby(al['CityTime ID']).transform('sum')
al['C Feb Sum'] = al['C Feb'].groupby(al['CityTime ID']).transform('sum')
al['C Mar Sum'] = al['C Mar'].groupby(al['CityTime ID']).transform('sum')
al['C Apr Sum'] = al['C Apr'].groupby(al['CityTime ID']).transform('sum')

al['C May Sum'] = al['C May'].groupby(al['CityTime ID']).transform('sum')
al['C Jun Sum'] = al['C Jun'].groupby(al['CityTime ID']).transform('sum')
al['C Jul Sum'] = al['C Jul'].groupby(al['CityTime ID']).transform('sum')
al['C Aug Sum'] = al['C Aug'].groupby(al['CityTime ID']).transform('sum')
al['C Sep Sum'] = al['C Sep'].groupby(al['CityTime ID']).transform('sum')
al['C Oct Sum'] = al['C Oct'].groupby(al['CityTime ID']).transform('sum')
al['C Nov Sum'] = al['C Nov'].groupby(al['CityTime ID']).transform('sum')
al['C Dec Sum'] = al['C Dec'].groupby(al['CityTime ID']).transform('sum')


al = al.drop_duplicates(['CityTime ID']).reset_index(drop=True)

#al = al[["CityTime ID", "Last Name", "First Name","Name", "Rank", "Title","Inspector","Leave Type",  "C Jan Sum", "C Feb Sum", "C Mar Sum", 'C Apr Sum','C May Sum','C Jun Sum', 'C Jul Sum', 'C Aug Sum', 'C Sep Sum', 'C Oct Sum' ]]
al = al[["CityTime ID", "Last Name", "First Name","Rank", "Title","Inspector", "C Jan Sum", "C Feb Sum", "C Mar Sum", 'C Apr Sum','C May Sum','C Jun Sum', 'C Jul Sum', 'C Aug Sum', 'C Sep Sum', 'C Oct Sum', 'C Nov Sum', 'C Dec Sum' ]]
            
al.to_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Enforcement Inspection Analysis\\output\\LEAVE INSPECTOR ANNUAL LEAVE JAN-DEC.csv")

# Part 7: CALCULATE AND EXPORT SICK LEAVE

In [14]:
sl = lv
            
sl = sl[sl['Leave Type'] == 'SICK LEAVE']
sl = sl.reset_index(drop=True)


#ADD TOTsl LEAVE HOURS PER PERSON AND THEN DROP DUPLICATES
sl['C Jan Sum'] = sl['C Jan'].groupby(sl['CityTime ID']).transform('sum')
sl['C Feb Sum'] = sl['C Feb'].groupby(sl['CityTime ID']).transform('sum')
sl['C Mar Sum'] = sl['C Mar'].groupby(sl['CityTime ID']).transform('sum')
sl['C Apr Sum'] = sl['C Apr'].groupby(sl['CityTime ID']).transform('sum')

sl['C May Sum'] = sl['C May'].groupby(sl['CityTime ID']).transform('sum')
sl['C Jun Sum'] = sl['C Jun'].groupby(sl['CityTime ID']).transform('sum')
sl['C Jul Sum'] = sl['C Jul'].groupby(sl['CityTime ID']).transform('sum')
sl['C Aug Sum'] = sl['C Aug'].groupby(sl['CityTime ID']).transform('sum')
sl['C Sep Sum'] = sl['C Sep'].groupby(sl['CityTime ID']).transform('sum')
sl['C Oct Sum'] = sl['C Oct'].groupby(sl['CityTime ID']).transform('sum')
sl['C Nov Sum'] = sl['C Nov'].groupby(sl['CityTime ID']).transform('sum')
sl['C Dec Sum'] = sl['C Dec'].groupby(sl['CityTime ID']).transform('sum')


sl = sl.drop_duplicates(['CityTime ID']).reset_index(drop=True)

sl = sl[["CityTime ID", "Last Name", "First Name","Rank", "Title","Inspector","C Jan Sum", "C Feb Sum", "C Mar Sum", 'C Apr Sum','C May Sum','C Jun Sum', 'C Jul Sum', 'C Aug Sum', 'C Sep Sum', 'C Oct Sum', 'C Nov Sum', 'C Dec Sum' ]]
            
sl.to_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Enforcement Inspection Analysis\\output\\LEAVE INSPECTOR SICK LEAVE JAN-DEC.csv")



In [ ]:
lv_unique = lv.groupby(["CityTime ID"])["Duration"].sum()
print len(lv_unique)
print len(lv)


# Part 8: CALCULATE  ALL TYPES OF LEAVE

In [15]:
#ADD TOTAL LEAVE HOURS PER PERSON AND THEN DROP DUPLICATES
lv['C Jan Sum'] = lv['C Jan'].groupby(lv['CityTime ID']).transform('sum')
lv['C Feb Sum'] = lv['C Feb'].groupby(lv['CityTime ID']).transform('sum')
lv['C Mar Sum'] = lv['C Mar'].groupby(lv['CityTime ID']).transform('sum')
lv['C Apr Sum'] = lv['C Apr'].groupby(lv['CityTime ID']).transform('sum')

lv['C May Sum'] = lv['C May'].groupby(lv['CityTime ID']).transform('sum')
lv['C Jun Sum'] = lv['C Jun'].groupby(lv['CityTime ID']).transform('sum')
lv['C Jul Sum'] = lv['C Jul'].groupby(lv['CityTime ID']).transform('sum')
lv['C Aug Sum'] = lv['C Aug'].groupby(lv['CityTime ID']).transform('sum')
lv['C Sep Sum'] = lv['C Sep'].groupby(lv['CityTime ID']).transform('sum')
lv['C Oct Sum'] = lv['C Oct'].groupby(lv['CityTime ID']).transform('sum')
lv['C Nov Sum'] = lv['C Nov'].groupby(lv['CityTime ID']).transform('sum')
lv['C Dec Sum'] = lv['C Dec'].groupby(lv['CityTime ID']).transform('sum')


lv = lv.drop_duplicates(['CityTime ID']).reset_index(drop=True)

lv = lv[["CityTime ID", "Last Name", "First Name","Rank", "Title","Inspector", "C Jan Sum", "C Feb Sum", "C Mar Sum", 'C Apr Sum','C May Sum','C Jun Sum', 'C Jul Sum', 'C Aug Sum', 'C Sep Sum', 'C Oct Sum', 'C Nov Sum', 'C Dec Sum' ]]

In [16]:
lv

,CityTime ID,Last Name,First Name,Rank,Title,Inspector,C Jan Sum,C Feb Sum,C Mar Sum,C Apr Sum,C May Sum,C Jun Sum,C Jul Sum,C Aug Sum,C Sep Sum,C Oct Sum,C Nov Sum,C Dec Sum
0,97567,MARSHALL,JUDY,Civilian,COMMUNITY ASSOC,0,63.00,33.25,156.00,147.00,70.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,97705,TRUELSEN,RAFAEL,Civilian,ASO INSPEC PLUM,1,10.00,0.00,16.00,19.50,49.25,26.00,8.00,91.00,8.00,24.00,16.00,0.00
2,98293,CARONE,RALPH,Civilian,INSP-LO PRS BLR,0,138.00,70.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,97832,PARCZEWSKI,KRZYSZTOF,Civilian,ASO INSPEC CONS,1,48.00,0.00,40.00,57.50,32.00,72.00,24.00,8.00,40.00,8.00,32.00,0.00
4,97536,OWOEYE,CAROL,Civilian,SECRETARY,0,133.00,119.00,98.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,97989,LOPEZ,TINA,Civilian,PRAA 1 & 2 N,0,21.00,46.00,14.57,14.00,36.25,22.00,35.00,42.00,7.00,28.00,21.00,35.75
6,97892,QUINLAND,EDWIN,Civilian,ASO INSPEC CONS,0,56.75,87.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,385218,KOTSIKONAS,MICHELLE,Civilian,ASO INSPEC CONS,0,80.50,48.50,111.00,42.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,97865,TURNER,ROBERT,Civilian,ASO INSPEC CONS,1,7.00,27.50,24.50,52.25,12.75,16.00,32.75,58.75,20.50,43.50,32.25,74.25
9,97309,PORAZZO,PAUL,Civilian,ASOC INSP-ELECT,1,8.00,15.00,17.00,35.00,14.00,16.25,39.75,24.00,48.00,42.00,8.00,16.25


In [ ]:
print len(lv)

In [ ]:
#SUBSET DATA TO GET ONLY RELEVANT VARIABLES


In [ ]:
lv.dtypes

# CALCUATE TOTAL HOURS, USING OVERTIME AND LEAVE

In [ ]:
lv.head(2)

In [ ]:
ot.head(2)

In [17]:
#Fill NaN with zeros
lv = lv.fillna(0)
ot = ot.fillna(0)

#strip white spaces
#lv = lv.rename(columns={'Employee_Name': 'Name'})
ot = ot.rename(columns={'Employee_Name': 'Name'})

lv = lv.rename(columns={'C Jan Sum': 'Jan'})
lv = lv.rename(columns={'C Feb Sum': 'Feb'})
lv = lv.rename(columns={'C Mar Sum': 'Mar'})
lv = lv.rename(columns={'C Apr Sum': 'Apr'})
lv = lv.rename(columns={'C May Sum': 'May'})
lv = lv.rename(columns={'C Jun Sum': 'Jun'})
lv = lv.rename(columns={'C Jul Sum': 'Jul'})
lv = lv.rename(columns={'C Aug Sum': 'Aug'})
lv = lv.rename(columns={'C Sep Sum': 'Sep'})
lv = lv.rename(columns={'C Oct Sum': 'Oct'})
lv = lv.rename(columns={'C Nov Sum': 'Nov'})
lv = lv.rename(columns={'C Dec Sum': 'Dec'})


lv = lv.rename(columns={'CityTime ID': 'CityTimeID'})


#ot = ot.rename(columns={'Jan-16': 'Jan'})
#ot = ot.rename(columns={'Feb-16': 'Feb'})
#ot = ot.rename(columns={'Mar-16': 'Mar'})
#ot = ot.rename(columns={'Apr-16': 'Apr'})
#ot = ot.rename(columns={'May-16': 'May'})
#ot = ot.rename(columns={'Jun-16': 'Jun'})
#ot = ot.rename(columns={'Jul-16': 'Jul'})
#ot = ot.rename(columns={'Aug-16': 'Aug'})


#lv["Name"] = lv["Name"].map(str.strip)
#ot["Name"] = ot["Name"].map(str.strip)

In [18]:
#CALCULATE TOTAL HOURS WITH LEAVE

#Turn chain assignments off:

pd.set_option('chained_assignment', None)


#TOTAL WORKDAYS	Jan-16	Feb-16	Mar-16	Apr-16	May-16	Jun-16	Jul-16
#Including holidays	19	20	23	21	21	22	20
#Hours (workdays* 8)	152	160	184	168	168	176	160


lv["TH Jan"] = 152.0
lv["TH Feb"] = 160.0
lv["TH Mar"] = 184.0
lv["TH Apr"] = 168.0
lv["TH May"] = 168.0
lv["TH Jun"] = 176.0
lv["TH Jul"] = 160.0
lv["TH Aug"] = 184.0
lv["TH Sep"] = 168.0
lv["TH Oct"] = 160.0
lv["TH Nov"] = 152.0
lv["TH Dec"] = 168.0

count = 0
for i in range(0, len(lv)):
    lv["TH Jan"][i] = lv["TH Jan"][i] - lv["Jan"][i]
    lv["TH Feb"][i] = lv["TH Feb"][i] - lv["Feb"][i]
            
    lv["TH Mar"][i] = lv["TH Mar"][i] - lv["Mar"][i]
    lv["TH Apr"][i] = lv["TH Apr"][i] - lv["Apr"][i]
            
    lv["TH May"][i] = lv["TH May"][i] - lv["May"][i]
    lv["TH Jun"][i] = lv["TH Jun"][i] - lv["Jun"][i]
            
    lv["TH Jul"][i] = lv["TH Jul"][i] - lv["Jul"][i]
            
    lv["TH Aug"][i] = lv["TH Aug"][i] - lv["Aug"][i]
    
    lv["TH Sep"][i] = lv["TH Sep"][i] - lv["Sep"][i]
    
    lv["TH Oct"][i] = lv["TH Oct"][i] - lv["Oct"][i]

    lv["TH Nov"][i] = lv["TH Nov"][i] - lv["Nov"][i]

    lv["TH Dec"][i] = lv["TH Dec"][i] - lv["Dec"][i]

            
    count+=1
            
print count

print "done"

1474
done


In [75]:
lv

,CityTimeID,Last Name,First Name,Rank,Title,Inspector,Jan,Feb,Mar,Apr,...,TH Feb,TH Mar,TH Apr,TH May,TH Jun,TH Jul,TH Aug,TH Sep,TH Oct,TH Nov
0,97567,MARSHALL,JUDY,Civilian,COMMUNITY ASSOC,0,63.00,33.25,156.00,147.00,...,126.75,28.00,21.00,98.00,176.00,160.00,184.00,168.00,160.00,152.00
1,97832,PARCZEWSKI,KRZYSZTOF,Civilian,ASO INSPEC CONS,1,48.00,0.00,40.00,57.50,...,160.00,144.00,110.50,136.00,104.00,136.00,176.00,128.00,152.00,128.00
2,97705,TRUELSEN,RAFAEL,Civilian,ASO INSPEC PLUM,1,10.00,0.00,16.00,19.50,...,160.00,168.00,148.50,118.75,150.00,152.00,93.00,160.00,136.00,144.00
3,97536,OWOEYE,CAROL,Civilian,SECRETARY,0,133.00,119.00,98.00,0.00,...,41.00,86.00,168.00,168.00,176.00,160.00,184.00,168.00,160.00,152.00
4,98293,CARONE,RALPH,Civilian,INSP-LO PRS BLR,0,138.00,70.00,0.00,0.00,...,90.00,184.00,168.00,168.00,176.00,160.00,184.00,168.00,160.00,152.00
5,97659,VIDAL,CHRISTINE,Civilian,AD MGR-NON MGRL,0,59.25,42.00,63.00,65.00,...,118.00,121.00,103.00,154.00,148.00,129.75,142.00,125.25,135.50,89.00
6,97710,NIMROD,SABINAH,Civilian,AD MGR-NON MGRL,0,133.00,140.00,98.00,14.50,...,20.00,86.00,153.50,142.50,159.50,139.00,163.00,150.25,103.50,123.25
7,345801,SAMARAS,TASOS,Civilian,INSP-ELECTRICAL,0,9.25,0.00,0.00,0.00,...,160.00,184.00,168.00,168.00,176.00,160.00,184.00,168.00,160.00,152.00
8,97533,ATTALLAH,MIREILLE,Civilian,AST PN EXAM BLG,0,71.50,10.00,8.50,37.75,...,150.00,175.50,130.25,143.98,166.00,124.75,96.25,124.25,138.00,142.00
9,97892,QUINLAND,EDWIN,Civilian,ASO INSPEC CONS,0,56.75,87.00,0.00,0.00,...,73.00,184.00,168.00,168.00,176.00,160.00,184.00,168.00,160.00,152.00


In [19]:
#CALCULATE TOTAL HOURS WITH OVERTIME

#Turn chain assignments off:

pd.set_option('chained_assignment', None)

count = 0
for i in range(0, len(lv)):
    for j in range(0, len(ot)):
        if lv["CityTimeID"][i] == ot["CityTime ID"][j]:
            lv["TH Jan"][i] = lv["TH Jan"][i] + ot["Jan"][j]
            lv["TH Feb"][i] = lv["TH Feb"][i] + ot["Feb"][j]
            
            lv["TH Mar"][i] = lv["TH Mar"][i] + ot["Mar"][j]
            lv["TH Apr"][i] = lv["TH Apr"][i] + ot["Apr"][j]
            
            lv["TH May"][i] = lv["TH May"][i] + ot["May"][j]
            lv["TH Jun"][i] = lv["TH Jun"][i] + ot["Jun"][j]
            
            lv["TH Jul"][i] = lv["TH Jul"][i] + ot["Jul"][j]
            
            lv["TH Aug"][i] = lv["TH Aug"][i] + ot["Aug"][j]

            lv["TH Sep"][i] = lv["TH Sep"][i] + ot["Sep"][j]
            
            lv["TH Oct"][i] = lv["TH Oct"][i] + ot["Oct"][j]
            
            lv["TH Nov"][i] = lv["TH Nov"][i] + ot["Nov"][j]
            
            lv["TH Dec"][i] = lv["TH Dec"][i] + ot["Dec"][j]


            #lv["TH Jan"][i] = 152.0 + ot["Jan"][i] - lv["Jan"][i]
            #print ""
            #print "i: ", i
            #print "j: ", j
            
            
            count+=1
            
print count

print "done"

1087
done


In [20]:
#CORRECT THE TOTAL HOURS IF NOT AN INSPECTOR, THESE FOLKS ONLY WORK 7 HOUR DAYS

for i in range(0, len(lv)): 
    if lv["Inspector"][i] == 0:
        lv["TH Jan"][i] = lv["TH Jan"][i] - 19
        lv["TH Feb"][i] = lv["TH Feb"][i] - 20

        lv["TH Mar"][i] = lv["TH Mar"][i] - 23
        lv["TH Apr"][i] = lv["TH Apr"][i] - 21

        lv["TH May"][i] = lv["TH May"][i] - 21
        lv["TH Jun"][i] = lv["TH Jun"][i] - 22

        lv["TH Jul"][i] = lv["TH Jul"][i] - 20

        lv["TH Aug"][i] = lv["TH Aug"][i] - 23

        lv["TH Sep"][i] = lv["TH Sep"][i] - 21

        lv["TH Oct"][i] = lv["TH Oct"][i] - 20

        lv["TH Nov"][i] = lv["TH Nov"][i] - 20
        
        lv["TH Dec"][i] = lv["TH Dec"][i] - 21

        

In [ ]:
print len(ot)

In [21]:
lv.to_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Enforcement Inspection Analysis\\output\\TOTAL HOURS DEC.csv")

In [ ]:
print lv["Name"][952]
print lv["TH Jan"][952]


# Old Code

# INSPECTORS

In [ ]:
#READ IN INSPECTOR NAME LIST
#ip = pd.read_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Enforcement Inspection Analysis\\Unit_Inspector.csv")
ip = pd.read_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Enforcement Inspection Analysis\\Aug 2016\\Performance_Charts2.csv")


In [ ]:
ip = ip[["CitytimeID", "Inspector Name", "Unit"]]
ip.head()

In [ ]:
ip = ip.fillna(0)


In [ ]:
ip.head(5)

In [ ]:
ip["CitytimeID"] = ip["CitytimeID"].astype(int)


In [ ]:
print len(ip)

In [ ]:
ip = ip.drop_duplicates(['CitytimeID']).reset_index(drop=True)

In [ ]:
print len(ip)

In [ ]:
#ip = ip.drop_duplicates(['Inspector Name']).reset_index(drop=True)

In [ ]:
ip["Inspector Name"] = ip["Inspector Name"].map(str.upper)
ip["Inspector Name"] = ip["Inspector Name"].map(str.strip)



#insert unit in the dataframe
count = 0

lv["Unit"] = ""

for i in range(0, len(ip)):
    for j in range(0, len(lv)):
        if ip["CitytimeID"][i] == lv["CityTime ID"][j]:
            lv["Unit"][j] = ip["Unit"][i]
            count+=1

print count

In [ ]:
ip.head()

In [ ]:
#create list from inspectors
IpList = []
for i in range(0, len(ip)):
    IpList.append(ip["CitytimeID"][i])

In [ ]:
len(ip)

In [ ]:
len(IpList)

In [ ]:
#df = pd.DataFrame({'A' : [5,6,3,4], 'B' : [1,2,3, 5]})

#print df

#a = [3, 6]

#print ""
#print df[df['A'].isin(a)]

ipHours = lv[lv['CityTime ID'].isin(IpList)]


In [ ]:
ipHours = ipHours.reset_index()


In [ ]:
len(ipHours)

In [ ]:
IpList[1]

In [ ]:
ipHours2 = ipHours[["CityTime ID", "Name", "Title", "Approval Org/Command", "TH Jan", "TH Feb", "TH Mar", "TH Apr", "TH May", "TH Jun", "TH Jul","TH Aug", "Unit"]]

In [ ]:
ipHours2.head(10)

In [ ]:
ipHours2.to_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Enforcement Inspection Analysis\\Aug 2016\\INSPECTOR ANALYSIS AUG.csv")

In [ ]:
#count_same_day+=1
            
            time1 = lv["Start Time"][i]
            time2 = lv["End Time"][i] # waited a few minutes before pressing enter
            elapsedTime = time2 - time1
            print ""
            print i
            print elapsedTime
            

            total_hours = elapsedTime.total_seconds()/3600.0
            
            print total_hours
            
            #correct for full days off, 8 for inspectors, 7 for regular employees
            if (lv["Inspector"][i] == 0 and lv["Duration"][i] == '7:00'):
                total_hours = total_hours - 1
                
            if (lv["Inspector"][i] == 1 and lv["Duration"][i] == '8:00'):
                print 'IN HERE'
                total_hours = 8.0
                print "corrected total hours ", total_hours
            
            lv["C Jan"][i] = total_hours





bn = pd.read_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Enforcement Inspection Analysis\\Inspector List\\Inspector Headcount (9-16-16).csv")

"""
pd.set_option('chained_assignment', None)


#bn["TITLE"] = bn["TITLE"].astype(str)

#bn["First Name"] = ""
#bn["Last Name"] =""
#bn["Inspector"] = 0


for i in range(0, len(bn)):
    #print i
    a = bn["Name"][i]
    a = a.split()
    bn["Last Name"][i] = a[0]
    if len(a) >1:
        bn["First Name"][i] = a[1]
        
        
bn["First Name"] = bn["First Name"].map(str.strip)
bn["Last Name"] = bn["Last Name"].map(str.strip)


print "length of bn", len(bn)

#bn = bn[bn.TITLE.str.contains("INSP") == True]
#print "length of bn", len(bn)

for i in range(0, len(bn)):
    if "INSP" in bn["TITLE"][i]:
        bn["Inspector"][i] = 1
        print bn["Last Name"][i] + " " + bn["First Name"][i]
        
bn = bn[bn["Inspector"] != 0]
#bn = bn[bn.STATUS.str.contains("Inactive") == False]
print "length of bn", len(bn)


bn = bn.reset_index(drop=True)


bn["Supervisor Inspector"] = 0

bn["First Name"] = bn["First Name"].str.replace(',', '')
bn["Last Name"] = bn["Last Name"].str.replace(',', '')


bn["TITLE"] = bn["TITLE"].str.replace('CHEIF', 'CHIEF')


for i in range (0, len(bn)):
    if ("CHIEF" in bn["TITLE"][i] or "SUPER" in bn["TITLE"][i]):
        bn["Supervisor Inspector"][i] = 1

"""